# OpenAI Chat Completions API
https://platform.openai.com/docs/overview  
https://platform.openai.com/docs/api-reference/chat  

배포된 openai의 api key를 .env의 OPENAI_API_KEY에 등록하여 사용합니다.

## Create chat completion 요청(Request) param 정리
### POST https://api.openai.com/v1/chat/completions

| Param                    | Type                     | Req | Default        | 설명                                                   |
| ------------------------ | ------------------------ | --: | -------------- | ---------------------------------------------------- |
| `model`                  | string                   |   ✅ | -              | 사용할 모델 ID                                            |
| `messages`               | array                    |   ✅ | -              | 대화 메시지 배열(이전 대화 포함). 각 항목은 `role`, `content` 등을 포함   |
| `temperature`            | number                   |     | -              | 샘플링 랜덤성(높을수록 다양/창의). 보통 `top_p`와 둘 중 하나만 조정          |
| `top_p`                  | number                   |     | -              | Nucleus sampling. 확률 질량 상위 `top_p` 범위에서 샘플링          |
| `max_completion_tokens`  | integer | null           |     | -              | 생성 토큰 상한(출력 토큰 + reasoning 토큰 포함)                    |
| `max_tokens`             | integer | null           |     | -              | **Deprecated**. `max_completion_tokens` 사용 권장        |
| `n`                      | integer                  |     | 1              | 생성할 응답 후보(choices) 개수                                |
| `stop`                   | string | string[] | null |     | null           | 최대 4개의 중단 시퀀스. 해당 문자열을 만나면 생성 중단(일부 모델 미지원 가능)       |
| `stream`                 | boolean                  |     | false          | true면 SSE로 스트리밍 응답                                   |
| `stream_options`         | object                   |     | -              | 스트리밍 옵션(예: `include_usage` 등)                        |
| `response_format`        | object                   |     | -              | 출력 형식 강제(예: `json_schema`, `json_object`)            |
| `tools`                  | array                    |     | -              | 모델이 호출 가능한 도구 목록(function tool 등)                    |
| `tool_choice`            | string | object          |     | `auto`(모델에 따라) | 도구 호출 방식 제어(`none`, `auto`, `required`, 특정 도구 강제 등)  |
| `functions`              | array                    |     | -              | **Deprecated**. `tools`로 대체                          |
| `function_call`          | string | object          |     | -              | **Deprecated**. `tool_choice`로 대체                    |
| `frequency_penalty`      | number                   |     | 0              | 반복 억제(같은 토큰/표현 반복을 줄임)                               |
| `presence_penalty`       | number                   |     | 0              | 새로운 주제/토큰 등장 유도(이미 나온 토큰에 페널티)                       |
| `logprobs`               | boolean | null           |     | false          | 토큰 로그확률 정보 포함 여부                                     |
| `top_logprobs`           | integer | null           |     | -              | logprobs 사용 시, 각 토큰에 대해 상위 N개 후보 로그확률                |
| `seed`                   | integer | null           |     | -              | 가능하면 결정적 결과를 유도(완전 보장 아님)                            |
| `user`                   | string                   |     | -              | **Deprecated**(문서상 대체 필드 안내)                         |
| `metadata`               | object                   |     | -              | 임의 메타데이터(추적/분류 목적)                                   |
| `store`                  | boolean | null           |     | false          | 결과 저장 여부(저장된 completion은 삭제 API로 삭제 가능)              |
| `audio`                  | object | null            |     | -              | 오디오 출력 설정(voice/format 등, 오디오 모달리티 사용 시)             |
| `modalities`             | array                    |     | -              | 출력 모달리티 지정(예: `["text"]`, `["audio"]`)               |
| `reasoning_effort`       | string                   |     | -              | reasoning 모델에서 추론 노력 수준 제어(모델 지원 시)                  |
| `service_tier`           | string                   |     | -              | 처리 티어 지정(auto/default/priority/flex 등, 계정/모델 정책에 따름) |
| `prediction`             | object                   |     | -              | Predicted Output 설정(미리 아는 출력이 많을 때 속도 최적화)           |
| `prompt_cache_key`       | string                   |     | -              | 프롬프트 캐시 키(성능/비용 최적화 목적)                              |
| `prompt_cache_retention` | string | number          |     | -              | 캐시 유지 정책(지원 범위는 문서/정책에 따름)                           |
| `web_search_options`     | object                   |     | -              | 웹 검색 도구 사용 시 옵션(컨텍스트 크기/위치 근사 등, 지원 허용 시)            |
| `verbosity`              | string                   |     | `medium`       | 출력 장황함 가이드(`low`/`medium`/`high`)                    |


In [4]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPEN_AI_API_KEY')
URL = "https://api.openai.com/v1/chat/completions"
model = "gpt-4o-mini"

### REST API 요청
라이브러리 없이 직접 HTTP 통신을 통해 api를 호출한다.

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}"
}

payload = {
    "model": model,
    "messages": [
        {"role": "system", "content": "당신은 친절한 AI 강사입니다."},
        {"role": "user", "content": "Chat Completions API가 뭐야? 2~3문장으로 답변해줘"}
    ]
}

response = requests.post(URL, headers=headers, json=payload)
pprint(response.json())
print(response.json()['choices'][0]['message']['content'])

### OpenAI SDK를 활용한 요청
공식 라이브러리를 사용하여 생산성을 높이는 표준 방식이다.  
`pip install openai` 를 통해 설치한다.

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": "Openai SDK를 사용하면 어떤 점이 좋아?"}
    ]
)

print(completion.choices[0].message.content)

### System Prompt 비교

동일한 질문에 대해 AI의 페르소나에 따라 답변이 어떻게 달라지는지 확인해 보자

In [ ]:
user_input = "집에 있는데도 집에 가고 싶을 때 조언 해줘."

personas = {
    "열정적인 셰프": "당신은 요리에 인생을 건 셰프입니다. 인생의 모든 이치를 요리 과정과 재료에 비유하여 설명하세요.",
    "엄격한 헬스 트레이너": "당신은 매우 엄격한 운동 전문가입니다. 강한 어조로 자기관리를 강조하며 답변하세요.",
    "지혜로운 판다": "당신은 대나무 숲에 사는 느긋하고 지혜로운 판다입니다. 느릿느릿하고 평화로운 말투로 조언을 건네세요."
}

for name, prompt in personas.items():
    print(f"--- [{name}] 버전 ---")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_input}
        ]
    )
    print(response.choices[0].message.content)
    print("\n")

### Temperature 비교

동일한 질문에 대해 temperature에 따라 답변이 어떻게 달라지는지 확인해 보자

In [ ]:
creative_topic = "운동화 브랜드의 새로운 슬로건을 5개 제안해줘. 단, '속도'나 '승리' 같은 뻔한 단어는 제외하고 아주 기발하게 작성해줘."
temperatures = [0.3, 0.8, 1.0, 1.3, 1.5, 1.6, 1.8]

for t in temperatures:
    print(f"### 설정값 (Temperature): {t} ###")
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": creative_topic}],
        temperature=t,
        max_completion_tokens=200, 
        timeout=15.0
    )
    print(response.choices[0].message.content)
    print("=" * 50)

In [ ]:
creative_topic = "우리집 강아지의 별명을 3개 지어줘."
temperatures = [0.3, 0.8, 1.0, 1.3, 1.5, 1.6, 1.8]

for t in temperatures:
    print(f"### 설정값 (Temperature): {t} ###")
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": creative_topic}],
        temperature=t,
        max_completion_tokens=200, 
        timeout=15.0
    )
    print(response.choices[0].message.content)
    print("=" * 50)

### `messages` 배열을 활용한 대화 맥락 유지 (Context Window)
Chat Completions API는 상태를 저장하지 않는(Stateless) 방식이므로, 이전 대화 내역을 리스트에 계속 누적해서 보내야 한다.

In [ ]:
def chat_without_memory(user_input):
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": user_input}
        ]
    )
    
    # 3. 모델의 답변을 기록에 추가 (이것이 맥락 유지의 핵심)
    answer = response.choices[0].message.content
    
    return answer

# 실습 테스트
print("Q1: 내 이름은 jun이야.")
print(f"A1: {chat_without_memory('내 이름은 jun이야')}\n")

print("Q2: 내 이름이 뭐라고?")
print(f"A2: {chat_without_memory('내 이름이 뭐라고?')}")

In [ ]:
# 대화 내역을 저장할 리스트 초기화
history = [
    {"role": "system", "content": "당신은 사용자의 이름을 기억하는 비서입니다."}
]

def chat_with_memory(user_input):
    # 1. 사용자 질문을 기록에 추가
    history.append({"role": "user", "content": user_input})
    
    # 2. 전체 기록을 API에 전송
    response = client.chat.completions.create(
        model=model,
        messages=history
    )
    
    # 3. 모델의 답변을 기록에 추가 (이것이 맥락 유지의 핵심)
    answer = response.choices[0].message.content
    history.append({"role": "assistant", "content": answer})
    
    return answer

# 실습 테스트
print("Q1: 내 이름은 jun이야.")
print(f"A1: {chat_with_memory('내 이름은 jun이야.')}\n")

print("Q2: 내 이름이 뭐라고?")
print(f"A2: {chat_with_memory('내 이름이 뭐라고?')}")

### Structured Outputs (구조화된 출력)
모델의 답변을 단순히 텍스트로 받는 것이 아니라, JSON 형태로 고정하여 받을 수 있다.  
웹 서비스의 백엔드에서 데이터를 바로 처리해야 할 때 필수적인 기능이다.  
여기서는 `JSON mode(json_object)`로 json format을 활용하지만,  
이후에는 pydantic 라이브러리를 활용한 `JSON Scheme` 방식을 통해 명확한 json 응답 형식을 지정한다.

In [ ]:
import json
from pprint import pprint 

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "너는 요리사야. 답변은 반드시 JSON 형식으로 해줘."},
        {"role": "user", "content": "떡볶이 레시피 알려줘."}
    ],
    # JSON 모드 활성화
    response_format={"type": "json_object"}
)

# 문자열로 온 답변을 직접 파싱해야 함
res_json = json.loads(response.choices[0].message.content)
print(res_json)

### Streaming (실시간 응답 처리)
stream=True 설정을 통해 활성화한다.  
서버는 SSE(Server-Sent Events) 프로토콜을 사용하여 응답을 끊지 않고 조각(Chunk) 단위로 지속적으로 전송한다.  
응답 객체는 제너레이터 형식으로, for 루프를 사용해 활용할 수 있다.

In [ ]:
prompt = "양자 역학에 대해 초등학생도 이해할 수 있게 설명해줘."
print(f"질문: {prompt}\n")
print("답변: ", end="")

response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
    stream=True 
)

full_response = ""
for chunk in response:
    content = chunk.choices[0].delta.content
    if content:
        print(content, end="", flush=True) # flush 옵션을 통해 출력 버퍼를 즉시 비워 스트리밍 답변이 지연 없이 실시간으로 표시되도록 한다.
        full_response += content

print("\n\n--- 스트리밍 종료 ---")

### 비동기 요청


In [ ]:
from openai import AsyncOpenAI
import asyncio

async_client = AsyncOpenAI(api_key=OPENAI_API_KEY)

async def get_food_recommendation(city):
    print(f"[{city}] 맛집 검색 시작...")
    response = await async_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"{city}에 가면 꼭 먹어야 할 음식 딱 한 가지만 추천해줘."}]
    )
    print(f"[{city}] 검색 완료!")
    return f"{city}: {response.choices[0].message.content}"

async def main():
    cities = ["서울", "파리", "뉴욕", "도쿄", "방콕", "로마"]
    tasks = [get_food_recommendation(c) for c in cities]
    
    # 여러 요청을 동시에(병렬로) 처리
    results = await asyncio.gather(*tasks)
    
    print("\n--- [여행자들을 위한 미식 가이드] ---")
    for r in results:
        print(r)

await main()

### Logprobs - 확률 확인하기

In [ ]:
import math

prompt = "새로 오픈한 조용한 북카페 이름을 한글로 딱 하나만 추천해줘."
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
    logprobs=True,
    top_logprobs=3,
    max_completion_tokens=50
)

content = response.choices[0].message.content
logprobs_data = response.choices[0].logprobs.content

print(f"질문: {prompt}")
print(f"답변: {content}\n")
print(f"{'Token':<15} | {'Probability':<12} | {'Top Alternatives'}")
print("-" * 60)

for lp in logprobs_data:
    prob = math.exp(lp.logprob) * 100
    alternatives = [f"{top.token}({math.exp(top.logprob)*100:.1f}%)" for top in lp.top_logprobs]
    print(f"{lp.token:<15} | {prob:>10.2f}% | {', '.join(alternatives)}")

